In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.contrib.keras import activations,layers,losses,optimizers,models,utils
from sklearn.preprocessing import MinMaxScaler
from collections import OrderedDict
%matplotlib inline

In [19]:
#Import the Data
df_train = pd.read_csv('./training.csv')
df_test = pd.read_csv('./sorted_test.csv')

In [20]:
y_train = df_train.filter(['SOC', 'pH', 'Ca', 'P', 'Sand'], axis=1)

X_train = df_train.drop(['SOC', 'pH', 'Ca', 'P', 'Sand'], axis=1)

In [21]:
SOC = y_train['SOC']
pH = y_train['pH']
Ca = y_train['Ca']
P = y_train['P']
Sand = y_train['Sand']

In [22]:
pd.value_counts(X_train['Depth'])

Topsoil    581
Subsoil    576
Name: Depth, dtype: int64

In [23]:
# Set Topsoil=0, Subsoil=1

In [24]:
X_train = X_train.replace(['Topsoil','Subsoil'],[0,1])
df_test = df_test.replace(['Topsoil','Subsoil'],[0,1])

In [25]:
# Only include numeric columns
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

X_train = X_train.select_dtypes(include=numerics)
X_test = df_test.select_dtypes(include=numerics)


In [26]:
# Normalize
scaler = MinMaxScaler()
scaled_x_train = scaler.fit_transform(X_train)
scaled_x_test = scaler.transform(X_test)

In [10]:
# Define the model
# Only one hidden layer network of layers=[3594,3594,1] running for 5 epochs.
# Hyperparameters can be changd anytime.

In [11]:
model = models.Sequential()
model.add(layers.Dense(units=3594,activation='relu',input_dim=3594))
# model.add(layers.Dense(units=3594,activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(units=3594,activation='relu'))
# model.add(layers.Dense(units=3594,activation='relu'))
model.add(layers.Dense(units=1,activation='linear'))

In [12]:
model.compile(optimizer='adam',loss='mse')


In [13]:
# Fit model to SOC and predict

model.fit(scaled_x_train,SOC,epochs=5)

SOC_pred = model.predict(scaled_x_test)

Epoch 1/5
1157/1157 [==============================] - 24s - loss: 141.6795    
Epoch 2/5
1157/1157 [==============================] - 24s - loss: 1.1338    
Epoch 3/5
1157/1157 [==============================] - 25s - loss: 0.9139    
Epoch 4/5
1157/1157 [==============================] - 22s - loss: 0.6599    
Epoch 5/5
1157/1157 [==============================] - 22s - loss: 0.5387    


In [14]:
# Fit model to SOC and predict

model.fit(scaled_x_train,pH,epochs=5)

pH_pred = model.predict(scaled_x_test)

Epoch 1/5
1157/1157 [==============================] - 22s - loss: 0.9994    
Epoch 2/5
1157/1157 [==============================] - 21s - loss: 0.9298    
Epoch 3/5
1157/1157 [==============================] - 20s - loss: 0.8268    
Epoch 4/5
1157/1157 [==============================] - 20s - loss: 0.7008    
Epoch 5/5
1157/1157 [==============================] - 21s - loss: 0.5494    


In [15]:
# Fit model to Ca and predict

model.fit(scaled_x_train,Ca,epochs=5)

Ca_pred = model.predict(scaled_x_test)

Epoch 1/5
1157/1157 [==============================] - 23s - loss: 0.8848    
Epoch 2/5
1157/1157 [==============================] - 22s - loss: 0.7660    
Epoch 3/5
1157/1157 [==============================] - 21s - loss: 0.5348    
Epoch 4/5
1157/1157 [==============================] - 20s - loss: 0.4212    
Epoch 5/5
1157/1157 [==============================] - 21s - loss: 0.5328    


In [16]:
# Fit model to P and predict

model.fit(scaled_x_train,P,epochs=5)

P_pred = model.predict(scaled_x_test)

Epoch 1/5
1157/1157 [==============================] - 20s - loss: 1.1557    
Epoch 2/5
1157/1157 [==============================] - 21s - loss: 1.0217    
Epoch 3/5
1157/1157 [==============================] - 23s - loss: 1.0084    
Epoch 4/5
1157/1157 [==============================] - 25s - loss: 1.0056    
Epoch 5/5
1157/1157 [==============================] - 27s - loss: 1.0016    


In [17]:
# Fit model to Sand and predict

model.fit(scaled_x_train,Sand,epochs=5)

Sand_pred = model.predict(scaled_x_test)

Epoch 1/5
1157/1157 [==============================] - 24s - loss: 0.9170    
Epoch 2/5
1157/1157 [==============================] - 21s - loss: 0.4906    
Epoch 3/5
1157/1157 [==============================] - 23s - loss: 0.4541    
Epoch 4/5
1157/1157 [==============================] - 24s - loss: 0.3978    
Epoch 5/5
1157/1157 [==============================] - 23s - loss: 0.3750    


In [37]:
# Reshape 'em
Ca_pred = Ca_pred.reshape([727,])
P_pred = P_pred.reshape([727,])
pH_pred = pH_pred.reshape([727,])
SOC_pred = SOC_pred.reshape([727,])
Sand_pred = Sand_pred.reshape([727,])

In [59]:
d = (('PIDN', df_test['PIDN']), ('Ca', Ca_pred), ('P', P_pred),('pH',pH_pred) , ('SOC',SOC_pred),('Sand',Sand_pred))
y_pred = pd.DataFrame(OrderedDict(d))

In [60]:
y_pred

,PIDN,Ca,P,pH,SOC,Sand
0,09gt9UK5,-0.396264,-0.400037,-0.304398,-0.978111,0.062810
1,0BVvxJ6a,-0.110512,-0.274317,0.396777,-0.633542,-0.447700
2,0KbdgApg,-0.240807,-0.284985,-0.052054,-0.238671,-0.524178
3,0MnuuduB,-0.126431,-0.085587,-0.037216,0.470117,-1.027744
4,0PcIsF9z,-0.434619,-0.380219,-0.527216,-0.819247,-0.025661
5,0ShTFLYJ,-0.357799,-0.519694,-0.012916,-1.112083,0.412417
6,0XZh9jka,-0.121030,0.002319,-0.140371,0.680027,-1.090316
7,0Y7Wegri,-0.241629,-0.174990,0.153387,-0.435622,0.237074
8,0dTqplIM,-0.023428,-0.027509,0.345609,-0.301882,-0.756059
9,0lNiZicC,-0.400960,-0.473695,-0.404141,-0.984160,-0.309719


In [53]:
sampe = pd.read_csv('./sample_submission.csv')
sampe

,PIDN,Ca,P,pH,SOC,Sand
0,09gt9UK5,0,0,0,0,0
1,0BVvxJ6a,0,0,0,0,0
2,0KbdgApg,0,0,0,0,0
3,0MnuuduB,0,0,0,0,0
4,0PcIsF9z,0,0,0,0,0
5,0ShTFLYJ,0,0,0,0,0
6,0XZh9jka,0,0,0,0,0
7,0Y7Wegri,0,0,0,0,0
8,0dTqplIM,0,0,0,0,0
9,0lNiZicC,0,0,0,0,0
